# Load data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

file = Path('../data/eeg/muse/subject0000/session001/recording_2021-07-15-11.32.37.PPG.csv')
df = pd.read_csv(file)
df['timestamps'] = pd.to_datetime(df['timestamps'],unit='s')
df.set_index('timestamps')

# Plot raw data

In [ ]:
df.plot('timestamps', 'PPG1')

In [ ]:
fs = 1 / np.mean([(t2 - t1).total_seconds() for t1, t2 in zip(df['timestamps'].iloc[:-1], df['timestamps'].iloc[1:])])

In [ ]:
from eegclassify.clean import filter
import scipy.signal

fs = 1 / np.mean([(t2 - t1).total_seconds() for t1, t2 in zip(df['timestamps'].iloc[:-1], df['timestamps'].iloc[1:])])
print(f"Sampling freq: {fs}")

def spectral(s: pd.Series):
    ppg = np.array([s.values.tolist()]).T
    ppg_filtered = filter(ppg, n_chans=1, low=30 / 60, high=200 / 60, sfreq=fs, verbose=False)
    f, Pxx = scipy.signal.welch(ppg_filtered.T, fs, nperseg=60*fs)
    f = 60 * f  # Convert to BPM
    return f, Pxx

In [ ]:
f, Pxx = spectral(df['PPG1'].iloc[0:int(60 * fs)])
minidx = np.argmax(f > 40)  # plot only certain frequencies
maxidx = np.argmax(f > 120)  # plot only certain frequencies
plt.plot(f[minidx:maxidx], Pxx.T[minidx:maxidx]);

# Pulse

Compute the mean pulse

In [ ]:
step = int(60 * fs)  # step has to be large enough to capture several heartbeats

pulse = []
s = df['PPG1']
for t in range(0, len(df), step):
    if t + step > len(s):
        continue
    f, Pxx = spectral(s.iloc[t:t+step])
    pulse += [f[np.argmax(Pxx)]]
    
pulse = np.array(pulse)
plt.plot(pulse)